# 인코더, 디코더, 트랜스포머 모델 구성

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Positional Encoding

class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    # 각도 배열 생성
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스에는 sin 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])
    # 배열의 홀수 인덱스에는 cosine 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    # sin과 cosine이 교차되도록 재배열
    pos_encoding = tf.stack([sines, cosines], axis=0)
    pos_encoding = tf.transpose(pos_encoding,[1, 2, 0]) 
    pos_encoding = tf.reshape(pos_encoding, [position, d_model])

    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [3]:
# Scaled dot product attention 

def scaled_dot_product_attention(query, key, value, mask):
  # 어텐션 가중치는 Q와 K의 닷 프로덕트
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 가중치를 정규화
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 패딩에 마스크 추가
  if mask is not None:
    logits += (mask * -1e9)  # 마스크 값이 1인 성분들에 매우 큰 음숫값을 더함 -> 소프트맥스를 거치면 0으로 수렴.

  # softmax적용
  attention_weights = tf.nn.softmax(logits, axis=-1) # row-wisely apply softmax

  # 최종 어텐션은 가중치와 V의 닷 프로덕트
  output = tf.matmul(attention_weights, value)
  return output

In [4]:
# Multi head attention

class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    # 각 어텐션 헤드는 "서로 다른" 가중치를 이용하여 Q, K, V 를 생성함을 상기
    # 서로 다른 어텐션 헤드들을 동시에 병렬로 늘어놓으면 결국 output 차원이 d_model 과 같아지므로
    # 병렬 처리 시 출력 차원이 d_model 임.
    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size): # batch_size : 한 번에 처리할 문장의 개수.
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth)) # 둘 째 차원은 seq_len
    return tf.transpose(inputs, perm=[0, 2, 1, 3]) # shape = (batch_size, num_heads, seq_len, depth)
    # 결론적으로 (batch, seq_len, d_model) |-> (batch, num_heads, seq_len, depth) 로 reshape 함.

  def call(self, inputs): # inputs 는 딕셔너리
    query, key, value, mask = inputs['query'], inputs['key'], inputs['value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # Q, K, V에 각각 Dense를 적용
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value) # 각각 shape = (batch, seq_len, d_model) 이 된다.

    # 병렬 연산을 위한 머리 여러 개
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size) # 각각 shape = (batch, num_heads, seq_len, depth)

    # 스케일드 닷 프로덕트 어텐션 적용
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)
        # shape = (batch, num_heads, seq_len, depth)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        # shape = (batch, seq_len, num_heads, depth)

    # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))
        # shape = (batch, seq_len, d_model)

    # 최종 결과에도 Dense를 한 번 더 적용
    outputs = self.dense(concat_attention)

    return outputs

In [5]:
def create_padding_mask(x): 
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  return mask[:, tf.newaxis, tf.newaxis, :]   # (batch_size, 1, 1, sequence length)

# x 에는 정수인코딩된 텐서가 들어갈 것임. 0 이 있는 위치에서 1, 나머지 위치에서 0을 반환.
# 잉여 차원 (... ,1, 1, ...) 을 끼워넣는 이유는 코드의 다른 부분에서 연산할 때
# tensorflow 의 broadcast 기능이 잘 작동하도록 맞춰주기 위함이다. GPT 답변 참조.

In [6]:
def create_look_ahead_mask(x): # 다음 단어를 가려주는 마스크
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

In [7]:
# 인코더 하나의 레이어를 함수로 구현.
# 인코더 레이어는 케라스 모델 클래스이며, [(... , d_model), padding mask] 를 인풋으로 받아 (... , d_model) 을 출력 

# 이 하나의 레이어 안에는 두 개의 서브 레이어가 존재 (멀티헤드 어텐션과 은닉층1개짜리 덴스레이어)
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })

  # 어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 두 번째 서브 레이어 : 2개의 완전연결층, 은닉층 차원은 units
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [8]:
# 인코더 : 임베딩, 포지션 인코딩, 그리고 여러 개 인코더레이어를 쌓아서 만들어지는 모델

def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # num_layers만큼 쌓아올린 인코더의 층.
  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [9]:
# 디코더 하나의 레이어를 함수로 구현.

# 이 하나의 레이어 안에는 세 개의 서브 레이어가 존재.
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })

  # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 두 번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행 (인코더-디코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })

  # 마스크드 멀티 헤드 어텐션의 결과는
  # Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 세 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [10]:
# 디코더 : 임베딩, 포지션 인코딩, 그리고 여러 개 디코더레이어를 쌓아서 만들어지는 모델

def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
    
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')

  # 패딩 마스크
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  # Dropout이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [11]:
# Transformer

def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
    
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더에서 패딩을 위한 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용.
  # 내부적으로 패딩 마스크도 포함.
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
  # 디코더에서 패딩을 위한 마스크
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더
  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  # 디코더
  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 완전연결층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

# 데이터셋을 로드하고 전처리

In [17]:
import pandas as pd

home_dir = os.path.expanduser("~")
file_path = os.path.join(home_dir, 'aiffel/transformer_chatbot/data/ChatbotData.csv')

chatbot_data = pd.read_csv(file_path)
print(chatbot_data.head())
print(chatbot_data.info())


                 Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB
None


In [18]:
# 챗봇을 만드는 상황이므로 감성 라벨은 필요 없다.
chatbot_data = chatbot_data[['Q', 'A']]
print(chatbot_data.tail())

                             Q                         A
11818           훔쳐보는 것도 눈치 보임.        티가 나니까 눈치가 보이는 거죠!
11819           훔쳐보는 것도 눈치 보임.             훔쳐보는 거 티나나봐요.
11820              흑기사 해주는 짝남.                    설렜겠어요.
11821  힘든 연애 좋은 연애라는게 무슨 차이일까?  잘 헤어질 수 있는 사이 여부인 거 같아요.
11822               힘들어서 결혼할까봐        도피성 결혼은 하지 않길 바라요.


In [22]:
def preprocess_sentence(sentence):
    # 1. 공백 제거
    sentence = sentence.strip()

    # 2. 단어와 구두점(. , ! ?) 사이에 공백 끼워넣기
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    # 3. 한글과 구두점을 제외한 모든 문자를 공백인 ' '로 대체
    sentence = re.sub(r"[^가-힣?.!,]+", " ", sentence)
    
    # 양쪽 공백 제거
    sentence = sentence.strip()

    return sentence

# 테스트
test_sentence = "안녕하세요!Chatbot을 만들어 보겠습니다. Let's start! 12345"
print(preprocess_sentence(test_sentence))

안녕하세요 ! 을 만들어 보겠습니다 . !


# 자연어처리를 위한 전처리

In [24]:
# Tensorflow Datasets 의 SubwordTextEncoder 를 tokenizer 로 사용하여 단어사전 생성 (각 단어를 정수로 인코딩)
import tensorflow_datasets as tfds
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(chatbot_data, target_vocab_size=2**13)

In [29]:
# 이때 디코더의 문장 생성 과정에서 사용할 '시작 토큰'과 '종료 토큰'에 대해서도 임의로 단어장에 추가하여서 정수로 사상. 
# 이미 생성된 단어장의 번호와 겹치지 않도록 각각 단어장의 크기와 그보다 1이 큰 수를 번호로 부여
# 시작 토큰과 종료 토큰에 고유한 정수를 부여.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

print('START_TOKEN의 번호 :' ,[tokenizer.vocab_size])
print('END_TOKEN의 번호 :' ,[tokenizer.vocab_size + 1])

# 시작 토큰과 종료 토큰을 고려하여 +2를 하여 단어장의 크기를 산정.
VOCAB_SIZE = tokenizer.vocab_size + 2
print(VOCAB_SIZE)

START_TOKEN의 번호 : [257]
END_TOKEN의 번호 : [258]
259


In [82]:
# 임의의 22번째 샘플에 대해서 정수 인코딩 작업을 수행.
# 각 토큰을 고유한 정수로 변환
print('100번째 질문 샘플: ' + chatbot_data['Q'][100])
print('정수 인코딩 후의 100번째 질문 샘플: {}'.format(tokenizer.encode(chatbot_data['Q'][100])))
print('100번째 답변 샘플: ' + chatbot_data['A'][100])
print('정수 인코딩 후의 100번째 답변 샘플: {}'.format(tokenizer.encode(chatbot_data['A'][100])))

100번째 질문 샘플: 거지됐어
정수 인코딩 후의 100번째 질문 샘플: [235, 178, 177, 237, 168, 129, 236, 145, 145, 237, 151, 181]
100번째 답변 샘플: 밥 사줄 친구를 찾아 보세요~
정수 인코딩 후의 100번째 답변 샘플: [236, 177, 166, 33, 237, 131, 173, 237, 165, 133, 33, 237, 186, 157, 235, 182, 173, 236, 166, 189, 33, 237, 177, 191, 237, 150, 133, 33, 236, 180, 181, 237, 133, 185, 237, 155, 149, 127]


In [87]:
tokenizer.decode([235, 178, 177, 237, 168, 129])

'거지'

In [91]:
#### 샘플의 최대 허용 길이 또는 패딩 후의 최종 길이
MAX_LENGTH = 40
print(MAX_LENGTH)

40


In [92]:
# 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []
  
  for (sentence1, sentence2) in zip(inputs, outputs):
    # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    # MAX_LENGTH 이하인 경우에만 데이터셋으로 허용
    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)
  
  # MAX_LENGTH 로 모든 데이터셋을 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_inputs, tokenized_outputs

In [93]:
# 정수 인코딩 과정을 수행하면서 샘플의 길이가 40을 넘는 경우는 샘플들을 필터링하였으므로 일부 샘플이 제외됨. 
# 단어장의 크기와 샘플의 개수를 확인.
questions, answers = tokenize_and_filter(chatbot_data['Q'], chatbot_data['A'])
print('단어장의 크기 :',(VOCAB_SIZE))
print('필터링 후의 질문 샘플 개수: {}'.format(len(questions)))
print('필터링 후의 답변 샘플 개수: {}'.format(len(answers)))

단어장의 크기 : 259
필터링 후의 질문 샘플 개수: 5497
필터링 후의 답변 샘플 개수: 5497


In [94]:
# 질문과 답변의 쌍을 tf.data.Dataset API의 입력으로 사용하여 파이프라인을 구성. 
# 이때, 교사 강요를 위해서 answers[:, :-1]를 디코더의 입력값, answers[:, 1:]를 디코더의 레이블로 사용.

BATCH_SIZE = 32
BUFFER_SIZE = 10000

# 디코더는 이전의 target을 다음의 input으로 사용.
# 이에 따라 outputs에서는 START_TOKEN을 제거.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [64]:
# num_layers, d-Model, units는 전부 사용자가 정할 수 있는 하이퍼파라미터 값.
# 논문에서 num_layers는 6, d-Model은 512였지만, 
# 빠르고 원활한 훈련을 위해 여기서는 각 하이퍼파라미터를 논문에서보다는 작은 값을 사용.

tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 2 # 인코더와 디코더의 층의 개수
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 256)    1120512     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

In [65]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

In [66]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [95]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [104]:
EPOCHS = 50
model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/50
172/172 [==============================] - 4s 25ms/step - loss: 0.5450 - accuracy: 0.5358
Epoch 2/50
172/172 [==============================] - 4s 25ms/step - loss: 0.5245 - accuracy: 0.5428
Epoch 3/50
172/172 [==============================] - 4s 25ms/step - loss: 0.5076 - accuracy: 0.5479
Epoch 4/50
172/172 [==============================] - 4s 25ms/step - loss: 0.4892 - accuracy: 0.5529
Epoch 5/50
172/172 [==============================] - 4s 26ms/step - loss: 0.4706 - accuracy: 0.5594
Epoch 6/50
172/172 [==============================] - 4s 26ms/step - loss: 0.4567 - accuracy: 0.5631
Epoch 7/50
172/172 [==============================] - 4s 26ms/step - loss: 0.4420 - accuracy: 0.5687
Epoch 8/50
172/172 [==============================] - 4s 26ms/step - loss: 0.4318 - accuracy: 0.5716
Epoch 9/50
172/172 [==============================] - 4s 26ms/step - loss: 0.4166 - accuracy: 0.5756
Epoch 10/50
172/172 [==============================] - 4s 26ms/step - loss: 0.4059 - accura

# 테스트

In [69]:
def decoder_inference(sentence):
  sentence = preprocess_sentence(sentence)

  # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
  # ex) Where have you been? → [[8331   86   30    5 1059    7 8332]]
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
  # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장. ex) 8331
  output_sequence = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 인퍼런스 단계
  for i in range(MAX_LENGTH):
    # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
    predictions = model(inputs=[sentence, output_sequence], training=False)
    predictions = predictions[:, -1:, :]

    # 현재 예측한 단어의 정수
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 예측한 단어들은 지속적으로 output_sequence에 추가됩니다.
    # 이 output_sequence는 다시 디코더의 입력이 됩니다.
    output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

  return tf.squeeze(output_sequence, axis=0)

In [70]:
# 임의의 입력 문장에 대해서 decoder_inference() 함수를 호출하여 챗봇의 대답을 얻는 sentence_generation() 함수를 만듭니다.
def sentence_generation(sentence):
  # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
  prediction = decoder_inference(sentence)

  # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('입력 : {}'.format(sentence))
  print('출력 : {}'.format(predicted_sentence))

  return predicted_sentence

In [105]:
sentence_generation('우울하다')

입력 : 우울하다
출력 : 울적..... 우울할땐... 울면


'울적..... 우울할땐... 울면'

In [106]:
sentence_generation('우울해')

입력 : 우울해
출력 : 울어도 충분해요.


'울어도 충분해요.'

In [107]:
sentence_generation('힘들다')

입력 : 힘들다
출력 : 힘들겠죠.


'힘들겠죠.'

In [108]:
sentence_generation('복잡한 문장으로 대답해봐')

입력 : 복잡한 문장으로 대답해봐
출력 : 일이라도 필요하겠죠.


'일이라도 필요하겠죠.'

In [109]:
sentence_generation('세 문장 이상으로 답해봐')

입력 : 세 문장 이상으로 답해봐
출력 : 자연스러운 현상이에요.


'자연스러운 현상이에요.'

In [111]:
sentence_generation('심리상담')

입력 : 심리상담
출력 : 무관심이 가보세요.


'무관심이 가보세요.'

In [112]:
sentence_generation('아무말')

입력 : 아무말
출력 : 아직 실감이 되길 바랍니다.


'아직 실감이 되길 바랍니다.'

In [113]:
sentence_generation('?')

입력 : ?
출력 : 저는 위로봇입니다.


'저는 위로봇입니다.'

In [114]:
sentence_generation('!')

입력 : !
출력 : 힘을 만살죄!


'힘을 만살죄!'

In [115]:
sentence_generation('.')

입력 : .
출력 : 이�나


'이�나'

In [116]:
sentence_generation('어떻게 할까')

입력 : 어떻게 할까
출력 : 답답하신가봐요.


'답답하신가봐요.'